# Qdrant Quick Demo

Simple collection creation and data insertion.


In [15]:
# Import Qdrant client
from qdrant_client import QdrantClient
import numpy as np

# Connect to local Qdrant instance
client = QdrantClient(url="http://localhost:6333")



In [ ]:
# Let's explore what's inside each ScoredPoint
print("🔍 Exploring ScoredPoint structure:")
print("=" * 50)

for i, result in enumerate(results):
    print(f"\n📊 Result {i+1}:")
    print(f"   Type: {type(result)}")
    print(f"   ID: {result.id}")
    print(f"   Score: {result.score}")
    print(f"   Payload: {result.payload}")
    print(f"   Vector length: {len(result.vector)}")
    
    # Access individual payload fields
    print(f"   Text: {result.payload['text']}")
    print(f"   Category: {result.payload['category']}")
    
    # Show first few vector dimensions
    print(f"   Vector sample: {result.vector[:5]}...")


In [ ]:
# Alternative ways to access ScoredPoint data
print("🛠️  Different ways to access ScoredPoint data:")
print("=" * 50)

# Method 1: Direct attribute access
first_result = results[0]
print(f"Method 1 - Direct access:")
print(f"   ID: {first_result.id}")
print(f"   Score: {first_result.score}")

# Method 2: Dictionary-style access for payload
print(f"\nMethod 2 - Payload access:")
print(f"   Text: {first_result.payload['text']}")
print(f"   Category: {first_result.payload['category']}")

# Method 3: Get all available attributes
print(f"\nMethod 3 - All attributes:")
print(f"   Available attributes: {dir(first_result)}")

# Method 4: Convert to dictionary (if needed)
print(f"\nMethod 4 - Convert to dict:")
result_dict = {
    'id': first_result.id,
    'score': first_result.score,
    'payload': first_result.payload,
    'vector_length': len(first_result.vector)
}
print(f"   Dict: {result_dict}")


In [ ]:
# Demonstrating different number formatting options
import random

# Sample scores (typical Qdrant similarity scores)
scores = [0.123456789, 0.987654321, 0.5, 0.000123456]

print("🔢 Number Formatting Examples:")
print("=" * 40)

for score in scores:
    print(f"Original: {score}")
    print(f"  :.3f   → {score:.3f}")      # 3 decimal places
    print(f"  :.2f   → {score:.2f}")      # 2 decimal places  
    print(f"  :.1f   → {score:.1f}")      # 1 decimal place
    print(f"  :.0f   → {score:.0f}")      # No decimal places
    print(f"  :.6f   → {score:.6f}")      # 6 decimal places
    print(f"  :.3%   → {score:.3%}")      # As percentage
    print(f"  :.3e   → {score:.3e}")      # Scientific notation
    print("-" * 30)


In [18]:
client.create_collection(
    collection_name="demo_collection",
    vectors_config ={'size':384,
                     'distance':"Cosine"}
)




True

In [20]:

# Generate sample vectors and metadata
points = [
    {
        "id": 1,
        "vector": np.random.randn(384).tolist(),  # Random 384-dim vector
        "payload": {"text": "Machine learning is awesome", "category": "AI"}
    },
    {
        "id": 2,
        "vector": np.random.randn(384).tolist(),  # Random 384-dim vector
        "payload": {"text": "Python is great for data science", "category": "Programming"}
    }
]



In [22]:
# Insert points into the collection
client.upsert(collection_name="demo_collection", points=points)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [23]:
# Search for similar vectors
query_vector = np.random.randn(384).tolist()  # Random query vector
results = client.search(
    collection_name="demo_collection",
    query_vector=query_vector,
    limit=2
)

/var/folders/p_/53rzshtx2lb_c4blf251m6d40000gn/T/ipykernel_23309/2666329683.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [24]:
# Display results
for result in results:
    print(f"Score: {result.score:.3f} | Text: {result.payload['text']}")


Score: -0.004 | Text: Machine learning is awesome
Score: -0.172 | Text: Python is great for data science


In [30]:
# 
for i in results:
    print(f"ID :{i.id}',  Score: {result.score:.3f}")
    print(f"Text: {result.payload['text']}")
    print(i)
# Every element has an id, score, payload and vector in it. 

ID :1',  Score: -0.172
Text: Python is great for data science
id=1 version=1 score=-0.0043940432 payload={'text': 'Machine learning is awesome', 'category': 'AI'} vector=None shard_key=None order_value=None
ID :2',  Score: -0.172
Text: Python is great for data science
id=2 version=1 score=-0.17185432 payload={'text': 'Python is great for data science', 'category': 'Programming'} vector=None shard_key=None order_value=None


In [31]:
high_similarity = [r for r in results if r.score > 0.5]
#→ “For each r in results, put r into the new list if r.score > 0.5.”